In [ ]:
!pip install mem0ai langchain

In [ ]:
!pip install -U langchain-groq

In [ ]:
from typing import List, Dict
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from mem0 import MemoryClient

Get your API key:
- https://app.mem0.ai/dashboard/api-keys
- https://console.groq.com/keys

In [ ]:
import os
os.environ["GROQ_API_KEY"] = ""
os.environ["MEM0_API_KEY"] = ""

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are a knowledgeable educational mentor AI. Use the provided context to guide students, suggest study strategies, and recommend resources for learning.
    Help answer questions about academic topics, provide explanations for complex concepts, and motivate students to achieve their goals.
    If you don’t have specific information, offer general advice on effective learning techniques and time management.
    Keep response brief and concise
    """),
    MessagesPlaceholder(variable_name="context"),
    HumanMessage(content="{input}")
])

In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile")
mem0 = MemoryClient(api_key=os.environ["MEM0_API_KEY"])

In [ ]:
def retrieve_context(query: str, user_id: str) -> List[Dict]:
    """Retrieve relevant context from Mem0"""
    memories = mem0.search(query, user_id=user_id)
    seralized_memories = ' '.join([mem["memory"] for mem in memories])
    context = [
        {
            "role": "system",
            "content": f"Relevant information: {seralized_memories}"
        },
        {
            "role": "user",
            "content": query
        }
    ]
    return context

def generate_response(input: str, context: List[Dict]) -> str:
    """Generate a response using the language model"""
    chain = prompt | llm
    response = chain.invoke({
        "context": context,
        "input": input
    })
    return response.content

def save_interaction(user_id: str, user_input: str, assistant_response: str):
    """Save the interaction to Mem0"""
    interaction = [
        {
          "role": "user",
          "content": user_input
        },
        {
            "role": "assistant",
            "content": assistant_response
        }
    ]
    mem0.add(interaction, user_id=user_id)

In [ ]:
def chat_turn(user_input: str, user_id: str) -> str:
    # Retrieve context
    context = retrieve_context(user_input, user_id)

    # Generate response
    response = generate_response(user_input, context)

    # Save interaction
    save_interaction(user_id, user_input, response)

    return response

In [ ]:
if __name__ == "__main__":
    print("Welcome to your EduTech! How can I assist you today?")
    user_id = "john"

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("EduTech Agent: Happy learning!")
            break

        response = chat_turn(user_input, user_id)
        print(f"EduTech Agent: {response}")

Welcome to your EduTech! How can I assist you today?
You: Tarun likes Physics. Harsha likes Maths
EduTech Agent: It seems like Tarun and Harsha have similar interests, with Tarun enjoying Physics and Harsha enjoying Maths. Both subjects are closely related and often complement each other. Tarun can explore how Maths is used in Physics, while Harsha can see the practical applications of Maths in Physics. This overlap can help deepen their understanding and appreciation of both subjects.
You: what is Tarun's favourite subject 
EduTech Agent: Tarun has two favourite subjects: Physics and Maths.
You: what about Harsha?
EduTech Agent: It seems like you didn't provide any context or information about Harsha. Could you please provide more details or clarify your question so I can better assist you?
You: quit
EduTech Agent: Happy learning!
